# Planet Isochrones - Mass Group Project 3
## Author:  Jason Wang (Northwestern University/CIERA)

Run the [SSW2024_PSF_Setup_Subtraction](https://colab.research.google.com/drive/1AmrmAj459MwUt4kkUbfquxnf1GkyO_F2?usp=share_link) notebook to download the data.  The setup notebook needs to just be run **once** once for Hands-on Session IV.

We will convert the flux ratio we measured for a planet into mass.

See the Group Project Idea 3 at the end of [SSW2024_ImagingExoplanets.ipynb](https://colab.research.google.com/drive/1XVUe9-qYAQs_9VVszxuw8Mih49dnhZjX?usp=sharing).

# Google Colab Usage
*Please read (don't just hit run) the information given above each code cell as there are separate install cells for Colab*
&#128992;
*and running Python on your computer*
&#128309;.

**Confirm login account**
* Please make sure to be logged in with the Google account you want to use for the exercises before running the code cells below. You can check by clicking the circular account icon in the top right corner of the colab notebook.

**Working directory**
* Note: The software and data will be installed in a directory called "SSW2024/PSF_Subtraction" in your Google drive. This directory will be created if it does not exist.

**Running cells**
* Run cells individually by clicking on the triangle on each cell

**To Restart runtime**
*   Click on Runtime menu item
*   Select Restart session
*   Select Run code cells individually from the top

**To Recreate runtime**
*   Click on Runtime menu item
*   Select Disconnect and Delete runtime
*   Select Run code cells individually from the top

**To Exit:**
*   Close the browser window

## &#128992; Install `cython` and `species` in Google Colab
**Run the cell below**

In [ ]:
!pip install cython
!pip install species

## &#128992; You need to restart your environment after running the previous commands

You may have been prompted already to Restart the Session.  If not, from the top menu: Runtime -> Restart session. After you have restarted the runtime, proceed.

We need to force an upgrade of typeguard to get `species` to work in the Colab environment. Otherwise the code will crash halfway though. Ignore pip's compatability warnings -- this is fine.

**Run the cell below**

In [ ]:
!pip install typeguard --upgrade

## &#128992; Setup Google Drive directory

#### &#128992; **Run the next cell to mount the Google Drive**

In [ ]:
# You will be prompted to Permit this notebook to access your Google Drive files - Click on "Connect to Google Drive"
# You will then be prompted to Choose an account - click on your preferred Google account
# You will then confirm that Google Drive for desktop wants to access your Google Account - scroll to click "Continue"
# You may get another prompt to allow additional access for this to work - scroll to click "Continue"

from google.colab import drive
drive.mount('/content/drive')

#### &#128992; **Run the next cell to define the ssw_dir**

In [ ]:
# PSF Subtraction
PSF_dir = 'SSW2024/PSF_Subtraction' #@param {type:"string"}

#### &#128992; **Run the next cell to change to the PSF_Subtraction directory**

In [ ]:
# change to the PSF Subtraction Directory
import os

# Google top level drive dir
drive_dir = "/content/drive/MyDrive/"

# ssw_dir directory path
PSF_path = os.path.join(drive_dir, PSF_dir)

# Change to the pRT_path
os.chdir(PSF_path)

## &#128309; Setup Python running on your own computer directory

**Run the Next cell**

In [ ]:
# Enter the full path to your PSF_Subtraction directory where you downloaded the data in the Setup notebbook

# For example:  '/Users/jsmith/PSF_Subtraction'
import os
absolute_path_to_the_data_folder_on_your_machine = '' # Please complete!
os.chdir(absolute_path_to_the_data_folder_on_your_machine)

# The rest of the notebook cells are for &#128992; Colab or &#128309; running Python on your computer

# Import `species` and setup your environment

You may get "errors" about MultiNest. You can safely ignore all of them.

If you are getting errors with a stacktrace indicating `species` did not load, try reloading the notebook and re-executing these cells.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import species
from species.data.database import Database
from species.read.read_isochrone import ReadIsochrone

%matplotlib inline

In [ ]:
species.SpeciesInit()

# Download evolutionary model

We will use the AMES-COND (Baraffe et al. 2003) model for this task. This is one of the classical models used for converting brightness to mass.

In [ ]:
database = Database()

In [ ]:
database.add_isochrones(model='ames')
read_iso = ReadIsochrone(tag='ames-cond')

# Plot curves of how bright an object is at 40 Myr as a function of mass

We will plot the L-band brightness (this is the wavelength band from the Keck data) of planets as a function of mass at 40 Myr (the age of the system).

Note that this plots the absolute magnitude in L-band (apparent magnitude at 10 pc)!

In [ ]:
iso_box = read_iso.get_isochrone(age=40.,
                                 masses=np.linspace(1., 30., 50),
                                 filter_mag="L'")
iso_box.open_box()

In [ ]:
plt.plot(iso_box.mass, iso_box.magnitude, label=f'Age = {iso_box.age} Myr')
plt.xlabel(r'Mass ($M_\mathrm{J}$)', fontsize=14)
plt.ylabel(r"$M_{L'}$", fontsize=14)
plt.legend(fontsize=14)
plt.show()

# Convert your measured L-band flux ratio to a planet absolute magnitude

Change the flux ratio to the one you measured.

In [ ]:
planet_fluxratio = 1e-4 # INSERT YOUR MEASURED FLUX RATIO HERE

star_lmag = 5.22
star_abs_lmag = star_lmag - 5 * np.log10(40.879/10)
planet_abs_lmag = star_abs_lmag - 2.5 * np.log10(planet_fluxratio)
print(planet_abs_lmag)

# Interpolate the evolutionary model to determine the mass of the planet

In [ ]:
planet_mass = np.interp(planet_abs_lmag, iso_box.magnitude[::-1], iso_box.mass[::-1])
print(planet_mass) # Jupiter masses